In [1]:
import numpy as np
import pandas as pd
import jscatter

In [2]:
n = 500
data = np.random.rand(n, 4)
data[:,3] = np.round(data[:,3] * 7).astype(int)

df = pd.DataFrame(
    data,
    columns=['x', 'y', 'value', 'group'],
)
df['group'] = df['group'].astype('int').astype('category').map(lambda c: chr(65 + c), na_action=None)
df.head(5)

,x,y,value,group
0,0.819162,0.801597,0.806771,G
1,0.717698,0.309595,0.495842,B
2,0.454103,0.817916,0.184145,C
3,0.964082,0.516785,0.311867,F
4,0.077463,0.251543,0.472809,F


In [3]:
scatterplot = jscatter.plot(
    data=df,
    x='x',
    y='y',
    color_by='value',
    color_order='reverse',
    size_by='value',
    opacity_by='value',
    connect_by='group',
    connection_color_by='value',
    connection_color_order='reverse',
    connection_size_by='value',
    height=600
)
scatterplot.show()

In [6]:
scatterplot.select(df.query('value > 0.9').index)

In [12]:
scatterplot.size = [1, 3, 60, 50, 100]